In [22]:
import cv2
import numpy as n
import tool_box
import time
from Arm_Lib import Arm_Device
from errorAnalyzer import ErrorAnalyzer

# 创建机械臂对象
Arm = Arm_Device()
# .1表示暂停0.1秒，也就是100毫秒。
time.sleep(.1)

In [23]:
import math

# def calculate_rotation_angle(point1, point2):
#     x1, y1 = point1[0],point1[1] # 机械臂坐标
#     x2, y2 = point2[0],point2[1] # 物体坐标

#     # 计算两个点之间的水平和垂直距离
#     dx = x2 - x1
#     dy = y2 - y1

#     # 计算旋转角度（弧度）
#     angle_rad = math.atan2(dy, dx)

#     # 将弧度转换为角度
#     angle_deg = math.degrees(angle_rad)

#     return angle_deg

# 定义夹积木块函数，enable=1：夹住，=0：松开
def arm_clamp_block(enable):
    if enable == 0:
        Arm.Arm_serial_servo_write(6, 60, 400)
    else:
        #Arm.Arm_serial_servo_write(6, 135, 400)
        Arm.Arm_serial_servo_write(6, 112, 400)# 112，夹子刚好可以夹紧物块
    time.sleep(.5)

# 定义移动机械臂函数,同时控制1-5号舵机运动，p=[S1,S2,S3,S4,S5]
def arm_move(p, s_time = 500):
    for i in range(5):
        id = i + 1
        if id == 5:
            time.sleep(.1)
            Arm.Arm_serial_servo_write(id,p[i],int(s_time * 1.2))
        else:
            Arm.Arm_serial_servo_write(id,p[i],s_time)
        time.sleep(.01)
    time.sleep(s_time/1000)

def control_Arm(id,angle,flag, wucha):
    if flag == False:
        angle = Arm.Arm_serial_servo_read(id) - (abs(angle) + wucha)
        time.sleep(1)
        Arm.Arm_serial_servo_write(id,angle,1500)
        time.sleep(1)
    else:
        angle = Arm.Arm_serial_servo_read(id) + (abs(angle) + wucha)
        time.sleep(1)
        Arm.Arm_serial_servo_write(id,angle,1500)
        time.sleep(1)
def arm_centrality():
    # 定义不同位置的变量参数
    p_mould = [90, 130, 0, 0, 90]
    # 让机械臂移动到一个准备抓取的位置
    arm_clamp_block(0)
    arm_move(p_mould, 1000)
    time.sleep(1)

# # 判断是否进行抓取
# def control_if_capture(ser):
#     _,b = tool_box.setial_communicatiuon_new(ser)
#     return b

In [42]:
arm_centrality() # 机械臂归中，移动到准备位置

In [49]:
# 必须的一些基本局部变量信息
Arm_Location = (0 ,0 ,0)
flag = False
flag_error = False
id = 1 # 舵机编号
z = 210 # 深度，以毫米为单位，手动量
wucha = 1 # 旋转角的误差，增加角度对准的准确率
error_range = 12 # 误差界限
num_judgments = 4 # 车的机械臂和四角机械臂不同的地方
color_information = "Red" # 模拟服务器发来的信息
flag_is_color = angle = coordinates = None
wait_time = 12 # 等待索尼镜头开启的等待时间

In [ ]:
# 打开串口,返回串口对象
ser = tool_box.open_serial()
# 生成误差分析对象
analyzer = ErrorAnalyzer(error_range, num_judgments)
# 启动串口
print("串口正在启动, 请等待{time}秒……".format(time=wait_time))
# 倒计时显示
for i in range(wait_time, 0, -1):
    print("\r", "倒计时{}秒！".format(i), end="", flush=True)
    time.sleep(1)
    
# 循环
while flag == False:
    if(ser.in_waiting > 0):
        result, flag = tool_box.setial_communicatiuon_mutiple(ser)
        if result is not None:
            flag, coordinates = tool_box.check_coloridx_imformation(result, color_information)
            # 对coordinates的第一位，也就是x值进行误差分析
            if flag_error == False:
                print("正在对获取的数据进行误差分析……")
                valid_data, flag_error = analyzer.analyze(coordinates[0])
                if flag_error:
                    print("误差分析已完毕，坐标已获取，信息如下：")
                    print(coordinates)
                    time.sleep(1)
                else:
                    flag = False #如果误差分析不合格，则继续获取数据
                    time.sleep(1)
        else:
            print("坐标数据不准确：",coordinates)
            # time.sleep(2)
            continue
    else:
        print("串口中无数据……")
        time.sleep(.5)

# 获取到目标的图像坐标，接下来转换成世界坐标
pixel_coords = coordinates
# pixel_coords = (232,62,40,40)
print("图像坐标：",pixel_coords)
# 图像是1280*960，但是我们的相机LCD显示的是320*240,所以要*4，将分辨率调回来，坐标才会准确 
image_points = tuple(x * 4 for x in pixel_coords)
# 计算世界坐标
color_x_point, color_y_point = tool_box.image_to_world_v2(z,image_points)
world_point = (color_x_point, color_y_point, z)
print(world_point)
# 计算旋转角度
angle = tool_box.azimuthangle(Arm_Location, world_point)
print("angle:",angle)
time.sleep(2)

if color_x_point > 0 and abs(angle) < 90:
    control_Arm(id, 90 - angle, False,wucha)
elif color_x_point < 0 and abs(angle) > 90:
    control_Arm(id, 90 - abs(angle), True,wucha)
ser.close() # 使用完毕后关闭串口

In [41]:
# 机械臂夹取
# 1、先伸直
angle_id_1 = Arm.Arm_serial_servo_read(1) # 读取第一舵机的角度
p_move_1 = [angle_id_1, 90, 0, 90, 90]
arm_move(p_move_1, 1500)# 机械臂伸直
time.sleep(2)
Arm.Arm_serial_servo_write(4,80,1500)
time.sleep(2)
Arm.Arm_serial_servo_write(2,53,1500)
time.sleep(2)
arm_clamp_block(1) # 夹取
time.sleep(1)
p_mould = [90, 130, 0, 0, 90]
arm_move(p_mould, 1000) # 回到指定位置

In [38]:
# 放到指定位置
p_mould = [90, 130, 0, 0, 90]

Arm.Arm_serial_servo_write(1,20,1500)
time.sleep(2)
angle_id_1 = Arm.Arm_serial_servo_read(1) # 读取第一舵机的角度
p_move_1 = [angle_id_1, 90, 0, 90, 90]
arm_move(p_move_1, 1500)# 机械臂伸直
time.sleep(2)
Arm.Arm_serial_servo_write(2,53,1000)
time.sleep(2)
Arm.Arm_serial_servo_write(4,80,1500)
time.sleep(2)
arm_clamp_block(0) # 放下物块
time.sleep(1)
arm_move(p_mould, 1000) # 回到指定位置

In [ ]:
del Arm  # 释放掉Arm对象

In [48]:
ser.close()